In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16, resnet50
from keras import models
from keras.models import Model
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims
import sklearn.preprocessing
from sklearn.metrics import accuracy_score
from matplotlib import pyplot


In [ ]:
%cd ..

/


### Loading and Creating Training dataset

In [ ]:


path = '/content/drive/MyDrive/CV/train_data/'
dirs = []
X = []
y = []
fnames = []
class_dict = {
    "Bed": 0,
    "Chair": 1,
    "Sofa": 2
}

for p,dir,fname in os.walk(path):
  dirs.append(dir)
  fnames.append(fname)
del fnames[0]
for i in dirs[0]:
  for p,dir,fname in os.walk(path+'/'+i+'/'):
    for j in fname:
      im = np.array((Image.open(path+'/'+i+'/'+j)))
      img = resize(im,(128,128))
      X.append(img)
      y.append(int(class_dict[i]))
 
    

In [ ]:
print("Number of Images in Training set:",len(X))

Number of Images in Training set: 284


### Creating Test dataset

In [ ]:
path = '/content/drive/MyDrive/CV/test_data/'
dirs = []
X_test = []
y_test = []
fnames = []

for p,dir,fname in os.walk(path):
  dirs.append(dir)
  fnames.append(fname)
del fnames[0]
for i in dirs[0]:
  for p,dir,fname in os.walk(path+'/'+i+'/'):
    for j in fname:
      im = np.array((Image.open(path+'/'+i+'/'+j)))
      img = resize(im, (128, 128))
      X_test.append(img)
      y_test.append(int(class_dict[i]))

In [ ]:
print("Number of Images in Test set:",len(X_test)) 

Number of Images in Test set: 16


### Performing Data Augmentation to improve the training sample

In [ ]:
datagen1 = ImageDataGenerator(horizontal_flip=True) #horizontal flip
datagen2 = ImageDataGenerator(rotation_range=90) #rotation 
datagen3 = ImageDataGenerator(height_shift_range=0.5) #height_shift
# datagen3 = ImageDataGenerator(zoom_range=[0.5,1.0])
Xa = []
ya = []

for i in range(len(X)):
  sample = expand_dims(X[i], 0)
  it1 = datagen1.flow(sample, batch_size=1)
  it2 = datagen2.flow(sample, batch_size=1)
  it3 = datagen3.flow(sample, batch_size=1)
  # it4 = datagen4.flow(sample, batch_size=1)
  im1 = it1.next()[0]
  im2 = it2.next()[0]
  im3 = it3.next()[0]
  # im4 = it4.next()
  
  ya.append(y[i])
  ya.append(y[i])
  ya.append(y[i])
  ya.append(y[i])

  Xa.append(im1)
  Xa.append(im2)
  Xa.append(im3)
  Xa.append(X[i])


In [ ]:
print("Number of Images after augmentation in Training set:",len(Xa))

Number of Images after augmentation in Training set: 1136


One hot encoding the categorical labels of the target class

In [ ]:
nClasses = 3
# Use Keras' handy utils

ya = tensorflow.keras.utils.to_categorical(ya, num_classes=nClasses)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes=nClasses)

Converting into NumPy Array

In [ ]:
ya=np.array(ya)
Xa=np.array(Xa)
X_test = np.array(X_test)
y_test = np.array(y_test)

Splitting the training data into training and validation data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(Xa, ya, test_size=1.0/8, stratify=ya, random_state=42)


In [ ]:
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)

In [ ]:
y_val_k[0]

1

## Classification Network

Building the classification network on top of the first 4 layers of VGG16 after unfreezing the last layer

In [ ]:
vgg_model = VGG16(weights='imagenet',include_top=False,input_shape=(128, 128, 3))
vgg_model.trainable = False
vgg_model.summary()

In [ ]:
model = models.Sequential()
model = Model(inputs = vgg_model.input, outputs = vgg_model.get_layer('block4_pool').output)

my_model = models.Sequential()
# my_model.add(K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224))))
my_model.add(model)

# my_model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
# my_model.add(MaxPooling2D((2,2), strides=(2,2)))


my_model.add(Flatten())

# my_model.add(Dense(units=512,activation="relu"))


my_model.add(Flatten())
my_model.add(K.layers.BatchNormalization())
my_model.add(Dense(units=256,activation="relu"))
my_model.add(K.layers.Dropout(0.5))

my_model.add(K.layers.BatchNormalization())
my_model.add(Dense(units=128,activation="relu"))
my_model.add(K.layers.Dropout(0.5))

my_model.add(Dense(units=3, activation="softmax"))
my_model.summary()

# model.add(MaxPool2D(pool_size=2, strides=2))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_6 (Functional)        (None, 4, 4, 512)         14714688  
                                                                 
 flatten_12 (Flatten)        (None, 8192)              0         
                                                                 
 flatten_13 (Flatten)        (None, 8192)              0         
                                                                 
 batch_normalization_10 (Bat  (None, 8192)             32768     
 chNormalization)                                                
                                                                 
 dense_17 (Dense)            (None, 256)               2097408   
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                     

In [ ]:
# opt = tensorflow.keras.optimizers.SGD(lr=0.0008)
opt = keras.optimizers.Adam(learning_rate=0.0001)
my_model.compile(loss='categorical_crossentropy', 
                           optimizer=opt, 
                           metrics=['accuracy'])

history_c = my_model.fit(X_train, y_train, batch_size=16, epochs=15, verbose=1, validation_data=(X_val, y_val))

Epoch 1/15
63/63 [==============================] - 7s 57ms/step - loss: 1.0961 - accuracy: 0.5644 - val_loss: 0.6122 - val_accuracy: 0.8521
Epoch 2/15
63/63 [==============================] - 2s 32ms/step - loss: 0.5698 - accuracy: 0.7757 - val_loss: 0.3949 - val_accuracy: 0.9155
Epoch 3/15
63/63 [==============================] - 2s 31ms/step - loss: 0.4027 - accuracy: 0.8330 - val_loss: 0.2820 - val_accuracy: 0.9296
Epoch 4/15
63/63 [==============================] - 2s 32ms/step - loss: 0.3274 - accuracy: 0.8632 - val_loss: 0.2047 - val_accuracy: 0.9577
Epoch 5/15
63/63 [==============================] - 2s 34ms/step - loss: 0.3244 - accuracy: 0.8813 - val_loss: 0.1629 - val_accuracy: 0.9507
Epoch 6/15
63/63 [==============================] - 2s 31ms/step - loss: 0.2463 - accuracy: 0.9115 - val_loss: 0.1258 - val_accuracy: 0.9577
Epoch 7/15
63/63 [==============================] - 2s 31ms/step - loss: 0.2104 - accuracy: 0.9235 - val_loss: 0.1004 - val_accuracy: 0.9718
Epoch 8/15
63

Saving the Model locally

In [ ]:
my_model.save("/content/drive/MyDrive/CV/models/vgg_model.h5")
print("Saved Model to disk")

Saved Model to disk


Loading the Model and calling the prediction on a single image

In [ ]:
from numpy import loadtxt
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/CV/models/vgg_model.h5')
path = "/content/drive/MyDrive/CV/test_data/Chair/Bridgeway Office Chair.jpg"
im = np.array((Image.open(path)))
img = resize(im,(128,128))
img = np.expand_dims(img, axis = 0)
output = model.predict(img).argmax(axis=-1)
print(output)

1/1 [==============================] - 0s 204ms/step
[2]
